# Tâche #1 : Classification d'incidents avec un réseau *feedforward* et des *embeddings* Spacy

On reprend la classification des descriptions d’accidents du premier travail. Le corpus de textes contient 3 partitions : 
-	un fichier d’entraînement -  data/incidents_train.json
-	un fichier de validation -  data/incidents_dev.json
-	un fichier de test - data/incidents_test.json

Entraînez un modèle de réseau de neurones de type feedforward multicouche (MLP) avec plongements de mots pour déterminer le type d’un incident à partir de sa description. 

Voici les consignes pour cette tâche : 

-	Nom du notebook : mlp.ipynb
-	Tokenisation : Utilisation de Spacy. 
-	Plongements de mots : Ceux de Spacy. 
-	Normalisation : Aucune normalisation. 
-	Agrégation des plongements de mots : Comparer les approches max, average et min pooling. 
-	Structure du réseau : 1 seule couche cachée dont vous choisirez la taille (à expliquer). 
-	Présentez clairement vos résultats et faites-en l’analyse. En cas de doute, inspirez-vous de ce qui a été fait dans le travail pratique #1. 

Vous pouvez ajouter au *notebook* toutes les cellules dont vous avez besoin pour votre code, vos explications ou la présentation de vos résultats. Vous pouvez également ajouter des sous-sections (par ex. des sous-sections 1.1, 1.2 etc.) si cela améliore la lisibilité.

Notes :
- Évitez les bouts de code trop longs ou trop complexes. Par exemple, il est difficile de comprendre 4-5 boucles ou conditions imbriquées. Si c'est le cas, définissez des sous-fonctions pour refactoriser et simplifier votre code. 
- Expliquez sommairement votre démarche.
- Expliquez les choix que vous faites au niveau de la programmation et des modèles (si non trivial).
- Analyser vos résultats. Indiquez ce que vous observez, si c'est bon ou non, si c'est surprenant, etc. 
- Une analyse quantitative et qualitative d'erreurs est intéressante et permet de mieux comprendre le comportement d'un modèle. 

## 1. Création du jeu de données (*dataset*)

In [ ]:
import json
import numpy as np
import spacy

from typing import List, Dict
from torch.utils.data import Dataset, DataLoader
from torch import FloatTensor, LongTensor
from typing import List, Callable
from spacy import Language
from torch import nn
from poutyne.framework import Experiment
from poutyne import set_seeds
from torch.optim import SGD
from torch.nn.functional import softmax


def load(filename: str):
    with open(filename, "r") as fp:
        incident_list = json.load(fp)
    texts, targets = zip(
        *[
            (incident.get("text"), int(incident.get("label")))
            for incident in incident_list
        ]
    )

    return list(texts), np.array(targets, dtype=np.uint8)


class SpacyDataset(Dataset):
    def __init__(
        self,
        dataset: List[str],
        target: np.array,
        sentence_aggregation_function: Callable[[List[str], Language], np.array] = None,
    ):
        self.dataset = dataset
        self.doc_embeddings = [None for _ in range(len(dataset))]
        self._sentence_aggregation_function = sentence_aggregation_function
        self._target = target

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        if self.doc_embeddings[index] is None:
            self.doc_embeddings[index] = self.sentence_aggregation_function(
                self.dataset[index]
            )
        return FloatTensor(self.doc_embeddings[index]), LongTensor(
            [self.target[index]]
        ).squeeze(0)

    def __iter__(self):
        return iter(self.dataset)

    @property
    def sentence_aggregation_function(self):
        return self._sentence_aggregation_function

    @sentence_aggregation_function.setter
    def sentence_aggregation_function(
        self, function: Callable[[List[str], Language], np.array]
    ):
        if not callable(function):
            raise ValueError("Parameter must be of function type")
        self._sentence_aggregation_function = function

    @property
    def target(self):
        return self._target

    @property
    def nb_classes(self):
        return len(set(self._target))

    @property
    def target_categories(self):
        return list(set(self._target))


texts, targets = load("./data/incidents_dev.json")
dev = SpacyDataset(texts, targets)
texts, targets = load("./data/incidents_train.json")
train = SpacyDataset(texts, targets)
texts, targets = load("./data/incidents_test.json")
test = SpacyDataset(texts, targets)

## 2. Gestion de plongements de mots (*embeddings*)

In [ ]:
nlp = spacy.load("en_core_web_md")


def average_embedding(sentence, nlp_model=nlp):
    tokenised_sentence = nlp_model(sentence)
    nb_column = len(tokenised_sentence)
    nb_rows = nlp_model.vocab.vectors_length
    sentence_embedding_matrix = np.zeros((nb_rows, nb_column))
    for index, token in enumerate(tokenised_sentence):
        sentence_embedding_matrix[:, index] = token.vector
    return np.average(sentence_embedding_matrix, axis=1)


def maxpool_embedding(sentence, nlp_model=nlp):
    tokenised_sentence = nlp_model(sentence)
    nb_column = len(tokenised_sentence)
    nb_rows = nlp_model.vocab.vectors_length
    sentence_embedding_matrix = np.zeros((nb_rows, nb_column))
    for index, token in enumerate(tokenised_sentence):
        sentence_embedding_matrix[:, index] = token.vector
    return np.max(sentence_embedding_matrix, axis=1)


def spacy_embedding(sentence, nlp_model=nlp):
    tokenised_sentence = nlp_model(sentence)
    return tokenised_sentence.vector

## Explications

### Choix de la taille de la couche cachée 


## 3. Création de modèle(s)

In [ ]:
train_dataloader = DataLoader(train, batch_size=16, shuffle=True)
dev_dataloader = DataLoader(dev, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test, batch_size=16, shuffle=True)

train.sentence_aggregation_function = spacy_embedding
dev.sentence_aggregation_function = spacy_embedding
test.sentence_aggregation_function = spacy_embedding

embedding_size = nlp.meta["vectors"]["width"]


class MultiLayerPerceptron(nn.Module):
    def __init__(self, input_size, hidden_layer_size, output_size):
        super().__init__()
        self.intput_layer = nn.Linear(input_size, hidden_layer_size)
        self.output_layer = nn.Linear(hidden_layer_size, output_size)

    def forward(self, x):
        x = self.intput_layer(x)
        x = nn.functional.relu_(x)
        x = self.output_layer(x)
        return x


set_seeds(42)
hidden_size = 100

directory_name = f"model/{spacy_embedding.__name__}_mlp"

model = MultiLayerPerceptron(embedding_size, hidden_size, train.nb_classes)
experiment = Experiment(directory_name, model, optimizer="SGD", task="classification")

## 4. Fonctions utilitaires

Vous pouvez mettre ici toutes les fonctions qui seront utiles pour les sections suivantes.

## 5. Entraînement de modèle(s)

In [ ]:
logging = experiment.train(
    train_dataloader, dev_dataloader, epochs=30, disable_tensorboard=True
)

## 6. Évaluation et analyse de résultats

In [ ]:
def get_most_probable_class(
    target: int,
    target_categories: List[str],
    aggregation_function: Callable[[str], List[str]],
    sentence: str,
    model: MultiLayerPerceptron,
):
    vectorized_sentence = aggregation_function(sentence)
    prediction = model(FloatTensor(vectorized_sentence).squeeze(0)).detach()
    output = softmax(prediction, dim=0)
    max_category_index = np.argmax(output)
    max_category = target_categories[max_category_index]
    print("\nClassification de la phrase: ", sentence)
    print("Sorties du réseau de neurones:", prediction)
    print("Valeurs obtenues après application de softmax:", output)
    print(
        f"Meilleure classe: {max_category} qui correspond en sortie à la neurone {max_category_index}"
    )
    print(f"Classe réelle : {target}")

    return int(max_category == target)

In [ ]:
def evaluate_predictions():
    total = 0
    correct = 0
    for i, sentence in enumerate(test):
        total += 1
        correct += get_most_probable_class(
            test.target[i],
            test.target_categories,
            test.sentence_aggregation_function,
            sentence,
            model,
        )
    print(correct, total)


display(test.target_categories)

In [ ]:
experiment.test(test_dataloader)

In [ ]:
evaluate_predictions()